# Parameter Extraction

In [1]:
from scipy.ndimage.interpolation import rotate, zoom
def rotateImage(img, angle, pivot):
    padX = [img.shape[1] - pivot[0,0], pivot[0,0]]
    padY = [img.shape[0] - pivot[0,1], pivot[0,1]]
    imgP = np.pad(img, [padY, padX], 'constant')
    imgR = ndimage.rotate(imgP, angle, reshape=False)
    return imgR[padY[0] : -padY[1], padX[0] : -padX[1]]



### labels: 0: noise, 1: iris, 2: AC, 3: cornea
def CalculateACArea(label1,showimg=False):
    label = copy.deepcopy(label1)
    label[label==1] = 0
    label[label==3] = 0
    label[label==2] = 1 # make AC 1 and others 0

    ### contour finding
    img = np.array(label*255,dtype=np.uint8)
    height, width = np.shape(img)
    contours,_ = cv2.findContours(img, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE) #find all contours
    cmax = max(contours, key = cv2.contourArea) # find maximum contour  
    imgnew = np.zeros( [height,width],dtype=np.uint8 )
    cv2.fillPoly(imgnew, pts =[cmax], color=(255)) #fill maximum contour and draw   
    
    # find left and right corner points
    p_left = (cmax[np.argmin(cmax[:,:,0])])
    p_right = (cmax[np.argmax(cmax[:,:,0])])
#     print(p_left,p_right)
    # rotate image wrt left corner so that both left and right become horizontal
    angle = np.rad2deg(np.arctan((p_right[0,1]-p_left[0,1])/(p_right[0,0]-p_left[0,0])))
    imgnew1 = rotateImage(imgnew,angle,p_left)
    imgnew1[imgnew1>125] = 255
    imgnew1[imgnew1<=125] = 0

    # get coutour again
    contours,_ = cv2.findContours(imgnew1, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)
    cmax = max(contours, key = cv2.contourArea) 
    p_left = (cmax[np.argmin(cmax[:,:,0])])
    p_right = (cmax[np.argmax(cmax[:,:,0])])
#     print(p_left,p_right)
    
    # get parameters
    center_lrcorner = int((p_left[0,0]+p_right[0,0])/2)
    ACdepth = int(np.sum(imgnew1[:,center_lrcorner]/255))
    ACwidth = int(np.sqrt((p_right[0,1]-p_left[0,1])**2+(p_right[0,0]-p_left[0,0])**2))
    ACarea  = int(np.sum(imgnew1/255))
    
    count =0
    for i in range(1000):
        pt1 = p_left[0,1] -i
        if imgnew1[pt1,center_lrcorner] == 0:
            count +=1
        if imgnew1[pt1,center_lrcorner] == 255:
            break
        if pt1 == 0:
            break
    LV = count  # lens vault
    
    #  AC depth near scleral spur : AOD
    AODrad = np.array([20,40,60,80,100])
    
    # left AOD
    left_AODarray=np.zeros(len(AODrad))
    for li in range(len(AODrad)):
        left_AOD = 0 # angle opening distance
        pt_array=[]  # store points just to check if it picked the correct points or not
        for i in range(AODrad[li]):
            pt_y = p_left[0,1]-i
            pt_x = int(np.sqrt(AODrad[li]**2-(p_left[0,1]-pt_y)**2)+p_left[0,0])
            pt_array.append([pt_x,pt_y])
            if imgnew1[pt_y,pt_x] == 255:
                left_AOD += 1    
        left_AODarray[li] = left_AOD
            
        
    # right AOD
    right_AODarray=np.zeros(len(AODrad))
    for li in range(len(AODrad)):
        right_AOD = 0 # angle opening distance
        pt_array=[]
        for i in range(AODrad[li]):
            pt_y = p_right[0,1]-i
            pt_x = int(-np.sqrt(AODrad[li]**2-(p_right[0,1]-pt_y)**2)+p_right[0,0])
            pt_array.append([pt_x,pt_y])
            if imgnew1[pt_y,pt_x] == 255:
                right_AOD +=1  
        right_AODarray[li] = right_AOD
    
    if showimg == True:
        plt.rcParams.update({'font.size': 12})
        f=plt.figure(figsize=(12,6))
        f.add_subplot(2, 2, 1)
        plt.imshow(label1)
        plt.title('Org image')
        f.add_subplot(2, 2, 2)
        plt.imshow(img)
        plt.title('Org. AC')
        f.add_subplot(2, 2, 3)
        plt.imshow(imgnew)
        plt.title('Cleaned AC')
        f.add_subplot(2, 2, 4)
        plt.imshow(imgnew1)
        plt.title('rotated AC')
        plt.colorbar()
        plt.grid(True)
        
    return ACwidth,ACdepth,LV,ACarea,left_AODarray,right_AODarray
    
    
def CalculateIrisArea(label1,showimg=False):
    label = copy.deepcopy(label1)
    label[label==2] = 0
    label[label==3] = 0 # make iris 1 and others 0

    img = np.array(label*255,dtype=np.uint8)
    width, height = np.shape(img)
    
    #find all contours
    contours,_ = cv2.findContours(img, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)
    # find maximum two contours 
    area_array = []
    for ii in range(len(contours)):
        area = cv2.contourArea(contours[ii])
        area_array.append(area)
    area_array = np.array(area_array)
    
    cmax1= contours[np.argmax(area_array)] # find first max contour
    area_array[area_array==np.max(area_array)] =0  
    cmax2= contours[np.argmax(area_array)]  # find second max contour

    #fill maximum contour and draw   
    imgnew = np.zeros( [width, height],dtype=np.uint8 )
    cv2.fillPoly(imgnew, pts =[cmax1,cmax2], color=(1))
    
    # rotate image wrt left corner so that both left and right become horizontal
    p_left1 = (cmax1[np.argmin(cmax1[:,:,0])])
    p_right1 = (cmax1[np.argmax(cmax1[:,:,0])])
#     print(p_left1,p_right1)
    cmax1_len = np.sqrt((p_left1[0,0]-p_right1[0,0])**2+(p_left1[0,1]-p_right1[0,1])**2)  # iris length
    
    p_left2 = (cmax2[np.argmin(cmax2[:,:,0])])
    p_right2 = (cmax2[np.argmax(cmax2[:,:,0])])
#     print(p_left2,p_right2)
    cmax2_len = np.sqrt((p_left2[0,0]-p_right2[0,0])**2+(p_left2[0,1]-p_right2[0,1])**2) # iris length

    iris_area = int(np.sum(imgnew))
    
    pupil_dia1 = np.sqrt((p_left1[0,0]-p_right2[0,0])**2+(p_left1[0,1]-p_right2[0,1])**2)
    pupil_dia2 = np.sqrt((p_right1[0,0]-p_left2[0,0])**2+(p_right1[0,1]-p_left2[0,1])**2)
    pupil_dia = pupil_dia1 if pupil_dia1 < pupil_dia2 else pupil_dia2
    
    
    if showimg == True:
        f=plt.figure(figsize=(12,6))
        f.add_subplot(2, 2, 1)
        plt.imshow(label1)
        plt.title('Org image')
        f.add_subplot(2, 2, 2)
        plt.imshow(img)
        plt.title('Org Iris')
        f.add_subplot(2, 2, 3)
        plt.imshow(imgnew)
        plt.title('Cleaned Iris')
        plt.grid(True)
        plt.colorbar()

    return iris_area, int(cmax1_len), int(cmax2_len), int(pupil_dia)

In [ ]:
# these are the 8 parameters 
# all these values are measured in pixels
# [318, 93, 0, 19765, 2028, 110, 112, 96] like this

ACwidth,ACdepth,LV,ACarea,left_AODarray,right_AODarray= CalculateACArea(label_tmp,showimg=True)
print(ACwidth,ACdepth,LV,ACarea,left_AODarray,right_AODarray)

iris_area, cmax1_len, cmax2_len, pupil_dia = CalculateIrisArea(label_tmp,showimg=True)
print(iris_area, cmax1_len, cmax2_len, pupil_dia)

# Pixel Measurement

In [1]:
from PIL import Image
import math
def pixelCount_left(label1,i_spur,j_spur):
    R = 100
    j_initial = j_spur-R
    pixCount = 0
    GREEN = (34, 255, 6, 255)
#     GREEN = label1[j_spur,i_spur]
    for j in range(j_initial,j_spur):
        i = i_spur-math.sqrt(R**2-((j-j_spur)**2))
        i_roundUp  = math.ceil(i)
        i_roundDown = math.floor(i)
#         print(j,i_roundUp,i_roundDown)
#         print(label1[j,i_roundUp])
        if(label1[j,i_roundUp] == GREEN or label1[j,i_roundDown] == GREEN):
            pixCount = pixCount+1
    return pixCount
def pixelCount_right(label1,i_spur,j_spur):
    R = 100
    j_initial = j_spur+R
    pixCount = 0
    GREEN = (34, 255, 6, 255)
#     GREEN = label1[j_spur,i_spur]
    for j in range(j_initial,j_spur,-1):
        i = i_spur-math.sqrt(R**2-((j-j_spur)**2))
        i_roundUp  = math.ceil(i)
        i_roundDown = math.floor(i)
        if(label1[j,i_roundUp] == GREEN or label1[j,i_roundDown] == GREEN):
            pixCount = pixCount+1
    return pixCount

In [2]:
image = Image.open("/Users/krishrajpal/Documents/P120_OS_21JUN17_020.labels.png")
width = image.width
height = image.height
# display width and height
print("The height of the image is: ", height)
print("The width of the image is: ", width)
label = image.load()
print(pixelCount_left(label,269,772))
print(pixelCount_right(label,214,128))

The height of the image is:  500
The width of the image is:  849
30
26


In [ ]:
conda install tensorflow

In [ ]:
import pickle
import joblib
from joblib import dump, load
from keras.models import load_models

model = load_model('Users/krishrajpal/Documents/BTP_MedicalSegmentation/Spec_segmentation_WithNetGenLabelImages_3.h5')
model.load_weights('Users/krishrajpal/Documents/BTP_MedicalSegmentation/best_weights_Spec_segmentation_WithNetGenLabelImages_3.hdf5')
print('Model Loaded successfully')